In [63]:
#GL_utils
from GL_utils import data_loading as GL_load
import importlib 
importlib.reload(GL_load)

#Module imports 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import re 
import seaborn as sns 
import os 


In [77]:
##Preprocessing##
#Read all input files into DataFrames 
formatted_dir = "formatted_input"
bpm_path = "{0}/19isolates_BPM.csv".format(formatted_dir)
mcseed_path = "{0}/19isolates_mcseed_pathwaycomplete.csv".format(formatted_dir)

COL_FILTERING_PATS = ["cecal","1C|2B"]
FILTERING_PATS_INCL_1B = ["cecal","1B|1C|2B"]

##2nd and 3rd trial merging## 
dc_tpm_path = "{0}/dc2ndadd_merged_tpm_data.tsv".format(formatted_dir) #Deprecated dataset 
rob_tpm_path = "{0}/rob3rd_merged_tpm_data.tsv".format(formatted_dir)
dc_count_path = "{0}/dc2ndadd_merged_count_data.tsv".format(formatted_dir)
rob_count_path = "{0}/rob3rd_merged_count_data.tsv".format(formatted_dir)
third_trial_counts_all_arms = GL_load.load_multiple_kallisto_pseudocounts([dc_count_path,rob_count_path],
                                                                col_stripping="est_counts",
                                                                 col_filtering_pats=FILTERING_PATS_INCL_1B)
third_trial_counts = GL_load.load_multiple_kallisto_pseudocounts([dc_count_path,rob_count_path],
                                                                col_stripping="est_counts",
                                                                 col_filtering_pats=COL_FILTERING_PATS)
third_trial_tpm_all_arms = GL_load.load_multiple_kallisto_pseudocounts([dc_tpm_path,rob_tpm_path],
                                                                       col_stripping="tpm",
                                                                        col_filtering_pats=FILTERING_PATS_INCL_1B)
##4th trial##
count_4th_trial_path = "{0}/4thtrial_merged_count_data.tsv".format(formatted_dir)
tpm_4th_trial_path = "{0}/4thtrial_merged_tpm_data.tsv".format(formatted_dir)
count_4t_df = GL_load.load_kallisto_pseudocounts(count_4th_trial_path,col_stripping="est_counts",col_filtering_pats=COL_FILTERING_PATS)
tpm_4t_df = GL_load.load_kallisto_pseudocounts(tpm_4th_trial_path,col_stripping="tpm",col_filtering_pats=COL_FILTERING_PATS)

##BPM and mcSEED annotations##
bpm_df = GL_load.load_BPM_df(bpm_path)
mcseed_df, pht_pathway_df = GL_load.load_mcSEED(mcseed_path)


#BPM Summary statistics 
print("BPM=1 pathways by strain")
print(bpm_df.sum())
print("Total BPM=1 pathways: {0}".format(bpm_df.sum().sum()))

BPM=1 pathways by strain
Isolate name
Bifidobacterium breve Bgsng463_m5_93            46
Bifidobacterium catenulatum Bgsng468_m22_84     44
Bifidobacterium longum infantis 40721_2D9_SN    50
Blautia luti Bg7063                             53
Blautia obeum Bg7063_SSTS2015                   53
Dorea formicigenerans Bg7063                    43
Dorea longicatena Bg7063                        45
Enterococcus_avium_Bang_SAM2_39_S1              57
Escherichia coli PS_131_S11                     77
Faecalibacterium prausnitzii Bg7063             43
Lactococcus garvieae Bang155_08_4B6_JG2017      32
Ligilactobacillus ruminis ATCC_25644            33
Mitsuokella multacida DSM_20544                 45
Prevotella copri PS_131_S11                     49
Prevotella stercorea DSM_18206                  29
Ruminococcus gnavus M8243_3A11_TMS_2014         63
Ruminococcus torques Bg7063                     50
Streptococcus gallolyticus PS_064_S07           42
Streptococcus pasteriuanus Bang_SAM2_39_S1  

In [78]:
tpm_cols_all = third_trial_tpm_all_arms.columns 
ordered_cols = [tpm_cols_all[tpm_cols_all.str.contains(pat)].to_list() for pat in ["1B","1C","2B"]]
ordered_cols = [item for sublist in ordered_cols for item in sublist] #Flatten from list of lists 
third_trial_tpm_all_arms = third_trial_tpm_all_arms.loc[:,ordered_cols]
third_trial_counts_all_arms = third_trial_counts_all_arms.loc[:,ordered_cols]

second_third_trial_full_dir = "formatted_output/2nd3rd_trial/full"
all_tpm_secondthird_fpath = "{0}/all_arms_full_merged_tpm.csv".format(second_third_trial_full_dir)
all_counts_secondthird_fpath = "{0}/all_arms_full_merged_count.csv".format(second_third_trial_full_dir)
third_trial_tpm_all_arms.to_csv(all_tpm_secondthird_fpath)
third_trial_counts_all_arms.to_csv(all_counts_secondthird_fpath)

# display(third_trial_tpm_all_arms)

In [66]:
#Assertion testing and visual inspection of DFs 
#assert(len(mcseed_df)==len(mcseed_df.index.unique())) #False - locus tags can have multiple entries in mcseed_df, corresponding
#to different subcomponents of same locus; must handle duplicate locus entries in tpm_df 
tpm_df = tpm_4t_df.copy()
count_df = count_4t_df.copy()
count_df_int = GL_load.int_round_counts(count_df,how="ceil")

show_tables = True 
if show_tables:
    display(mcseed_df)
    display(count_df)
    display(bpm_df)
    
full_dir_path = "formatted_output/4th_trial/full"
if not os.path.exists(full_dir_path):
    os.makedirs(full_dir_path)
overwrite_files = False
if overwrite_files:
    tpm_df.to_csv("{0}/full_merged_tpm.csv".format(full_dir_path))
    count_df.to_csv("{0}/full_merged_count.csv".format(full_dir_path))
    count_df_int.to_csv("{0}/full_merged_count_int.csv".format(full_dir_path))


,Isolate name,Protein name,Protein product,Functional category,Functional pathway,Phenotype,Strain
Locus tag,,,,,,,
ANCJAENF_00011,Bifidobacterium breve Bgsng463_m5_93,MalE,"Maltose/maltodextrin ABC transporter, substrat...",Carbohydrate utilization,maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,Bbr
ANCJAENF_00013,Bifidobacterium breve Bgsng463_m5_93,MalF,"Maltose/maltodextrin ABC transporter, permease...",Carbohydrate utilization,maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,Bbr
ANCJAENF_00014,Bifidobacterium breve Bgsng463_m5_93,MalG,"Maltose/maltodextrin ABC transporter, permease...",Carbohydrate utilization,maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,Bbr
ANCJAENF_00052,Bifidobacterium breve Bgsng463_m5_93,GalE,UDP-glucose 4-epimerase (EC 5.1.3.2),Carbohydrate utilization,galactose utilization; lactose utilization,Gal; Lac,Bbr
ANCJAENF_00063,Bifidobacterium breve Bgsng463_m5_93,GalE,UDP-glucose 4-epimerase (EC 5.1.3.2),Carbohydrate utilization,galactose utilization; lactose utilization,Gal; Lac,Bbr
...,...,...,...,...,...,...,...
LDOIJNDB_02225,Streptococcus pasteriuanus Bang_SAM2_39_S1,TreB_c,"PTS system, trehalose-specific IIC component (...",Carbohydrate utilization,trehalose utilization,Tre,Spa
LDOIJNDB_02226,Streptococcus pasteriuanus Bang_SAM2_39_S1,TreB_b,"PTS system, trehalose-specific IIB component (...",Carbohydrate utilization,trehalose utilization,Tre,Spa
LDOIJNDB_02245,Streptococcus pasteriuanus Bang_SAM2_39_S1,GalE,UDP-glucose 4-epimerase (EC 5.1.3.2),Carbohydrate utilization,galactose utilization; lactose utilization,Gal; Lac,Spa


,Pup_1-cecal_contents_53_1C_Pup_1,Pup_1-cecal_contents_54_2B_Pup_1,Pup_2-cecal_contents_53_1C_Pup_2,Pup_2-cecal_contents_54_2B_Pup_2,Pup_3-cecal_contents_53_1C_Pup_3,Pup_3-cecal_contents_54_2B_Pup_3,Pup_4-cecal_contents_53_1C_Pup_4,Pup_4-cecal_contents_54_2B_Pup_4,Pup_5-cecal_contents_53_1C_Pup_5,Pup_5-cecal_contents_54_2B_Pup_5,Pup_6-cecal_contents_53_1C_Pup_6,Pup_6-cecal_contents_54_2B_Pup_6,Pup_7-cecal_contents_53_1C_Pup_7,Pup_7-cecal_contents_54_2B_Pup_7,Pup_8-cecal_contents_53_1C_Pup_8
target_id,,,,,,,,,,,,,,,
ANCJAENF_00001,31592.600000,15244.40000,9574.71000,418.090000,28120.200000,427.762000,6.063660e+04,10.47990,6.397330e+04,10404.10000,5.238820e+04,20874.30000,30190.800000,20041.30000,47615.700000
ANCJAENF_00002,614048.000000,231644.00000,207898.00000,5709.760000,766224.000000,8698.240000,1.138520e+06,200.07000,1.178570e+06,185254.00000,1.196780e+06,363192.00000,493882.000000,319494.00000,893343.000000
ANCJAENF_00003,31592.600000,15244.40000,9574.71000,418.090000,28120.200000,427.762000,6.063660e+04,10.47990,6.397330e+04,10404.10000,5.238820e+04,20874.30000,30190.800000,20041.30000,47615.700000
ANCJAENF_00004,614048.000000,231644.00000,207898.00000,5709.760000,766224.000000,8698.240000,1.138520e+06,200.07000,1.178570e+06,185254.00000,1.196780e+06,363192.00000,493882.000000,319494.00000,893343.000000
ANCJAENF_00005,19.000000,3.00000,2.00000,0.000000,9.000000,0.000000,8.000000e+00,0.00000,2.000000e+00,15.00000,4.000000e+00,0.00000,7.000000,5.00000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LDOIJNDB_02259,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000e+00,0.00000,0.000000e+00,0.00000,0.000000,0.00000,0.000000
LDOIJNDB_02260,0.285714,1.85714,1.28571,0.857143,0.857143,0.285714,8.571430e-01,1.14286,4.285710e-01,7.85714,4.285710e-01,2.14286,0.571429,1.71429,0.571429
LDOIJNDB_02261,5997.760000,45761.30000,21327.30000,5092.810000,8685.180000,11591.400000,9.143430e+03,17391.40000,1.073310e+04,22426.50000,1.106550e+04,145913.00000,13167.100000,44534.00000,25791.600000


Isolate name,Bifidobacterium breve Bgsng463_m5_93,Bifidobacterium catenulatum Bgsng468_m22_84,Bifidobacterium longum infantis 40721_2D9_SN,Blautia luti Bg7063,Blautia obeum Bg7063_SSTS2015,Dorea formicigenerans Bg7063,Dorea longicatena Bg7063,Enterococcus_avium_Bang_SAM2_39_S1,Escherichia coli PS_131_S11,Faecalibacterium prausnitzii Bg7063,Lactococcus garvieae Bang155_08_4B6_JG2017,Ligilactobacillus ruminis ATCC_25644,Mitsuokella multacida DSM_20544,Prevotella copri PS_131_S11,Prevotella stercorea DSM_18206,Ruminococcus gnavus M8243_3A11_TMS_2014,Ruminococcus torques Bg7063,Streptococcus gallolyticus PS_064_S07,Streptococcus pasteriuanus Bang_SAM2_39_S1
Glc,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
Gal,0,0,1,0,0,1,1,1,1,1,0,0,0,1,1,0,0,0,1
Fru,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
Man,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,1,1
Tag,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lys_d,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Met_d,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Pro_d,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Thr_d,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0


In [67]:
#Exploratory analysis - comparing mcSEED annotated genes to merged_counts 

def reads_distribution_QC(count_df, mcseed_df,cecal_filter=True,pco_col_pat='1C',no_pco_col_pat='2B'):
    bpm1_annotated_count_loci = count_df.loc[count_df.index.isin(mcseed_df.index)]
    if cecal_filter: 
        bpm1_annotated_count_loci = bpm1_annotated_count_loci.loc[:,bpm1_annotated_count_loci.columns.str.contains('cecal')]
        
    bpm_counts_total_entries = len(bpm1_annotated_count_loci)*len(bpm1_annotated_count_loci.columns)
    #BPM mapping and zeroes in all samples
    #1. Total BPM1 loci; 2. fraction of total input loci; 3. hard-coded retained BPM1 loci (based on downstream analysis)
    #4. Zero-expression in all samples 
    print("Total BPM1 mapped loci: {0}".format(len(bpm1_annotated_count_loci)))
    print("BPM1 loci as fraction of total counts: {:.2%}".format(len(bpm1_annotated_count_loci)/len(count_df)))
    print("Retained BPM1 loci: {:.2%}".format(1980/len(bpm1_annotated_count_loci)))
    print("Transcripts with 0 expression in all samples: {0}".format(len(bpm1_annotated_count_loci.loc[bpm1_annotated_count_loci.mean(axis=1)==0])))
    print("")
    #Pco vs no pco sample partitioning; zero-fraction of reads in all, then Pco, no pco partitions
    pco_bpm1_counts= bpm1_annotated_count_loci.loc[:,bpm1_annotated_count_loci.columns.str.contains(pco_col_pat)]
    no_pco_bpm1_counts= bpm1_annotated_count_loci.loc[:,bpm1_annotated_count_loci.columns.str.contains(no_pco_col_pat)]
    zero_fraction_alldata = sum(bpm1_annotated_count_loci.values.flatten()==0)/(len(bpm1_annotated_count_loci)*len(bpm1_annotated_count_loci.columns))
    pco_zero_fraction = sum(pco_bpm1_counts.values.flatten()==0)/(len(pco_bpm1_counts)*len(pco_bpm1_counts.columns))
    no_pco_zero_fraction = sum(no_pco_bpm1_counts.values.flatten()==0)/(len(no_pco_bpm1_counts)*len(no_pco_bpm1_counts.columns))
    print("0 expression as fraction of total dataset: {:.2%}".format(zero_fraction_alldata))
    print("0 expression as fraction of P.copri samples: {:.2%}".format(pco_zero_fraction))
    print("0 expression as fraction of No P.copri samples: {:.2%}".format(no_pco_zero_fraction))
    
    #feature means and log transformed histograms 
    PSEUDOCOUNT = 10**-3
    bpm1_feature_means = bpm1_annotated_count_loci.mean(axis=1)
    log2_feature_means = np.log2(bpm1_feature_means+PSEUDOCOUNT)
    log2_feature_means.replace([np.inf, -np.inf], np.nan, inplace=True)
    print ("===Transcript count means BPM 1 transcripts===")
    fig,ax = plt.subplots(1,1,figsize=(6,6))
    sns.histplot(log2_feature_means,bins=30)
    ymin,ymax = ax.get_ylim()
    ax.vlines(4,ymin,ymax,color='black',linestyles='dashed')
    ax.set_ylim(ymin,ymax)
    ax.set_title("Log2 feature means")
    print("Fraction of Features with avg log2 expression > 4: {:.2%}".format(len(log2_feature_means[log2_feature_means>4])/len(log2_feature_means)))

    fig,ax = plt.subplots(1,1,figsize=(6,6))
    print ("===Transcript counts all data, BPM 1 transcripts===")
    log2_bpm1_counts  = np.log2(bpm1_annotated_count_loci+PSEUDOCOUNT)
    log2_bpm1_counts.replace([np.inf, -np.inf], np.nan, inplace=True)
#     assert(sum(log2_bpm1_counts.isna().values.flatten()))

    sns.histplot(log2_bpm1_counts.values.flatten(),bins=30,binrange=(-10,20))
    ymin,ymax = ax.get_ylim()
    ax.vlines(4,ymin,ymax,color='black',linestyles='dashed')
    ax.set_ylim(ymin,ymax)
    ax.set_title("Log2 counts distribution")
    
# reads_distribution_QC(count_df_int,mcseed_df,pco_col_pat='1C',no_pco_col_pat='2B')
# reads_distribution_QC(count_df,mcseed_df,pco_col_pat='1C',no_pco_col_pat='2B')


In [71]:
# cecal_abundance_path = "formatted_input/cecal_abundance.csv"
# cecal_abundance_df = pd.read_csv(cecal_abundance_path,sep=",",index_col="MouseID")
# cecal_abundance_df.columns = [col.strip() for col in cecal_abundance_df.columns]

display(cecal_abundance_df)

,Treatment,Mouse Number,Sex,B. breve Bgsng463.m5.93,B. catenulatum Bgsng463.m5.93,B. longum subsp. Infantis Bg2D9,B. longum subsp. Infantis Bg463,B. luti Bg7063,B. obeum Bg7063,D. formicigenerans Bg7063,...,L. garvieae Bang155.08_4B6_JG2017,L. ruminis ATCC 25644,M. multacida DSM 20544,P. copri PS131.S11,P. stercorea DSM 18206,R. gnavus M8243_3A11_TMS_2014,R. torques Bg7063,S. gallolyticus PS.064.S07,S. pasteriuanus Bang_SAM2.39.S1,Total Bacterial Load
MouseID,,,,,,,,,,,,,,,,,,,,,
Pup_1-cecal_contents_53_1C_Pup_1,Arm 1 (Pre-weaning P. copri colonization),1,female,5.96,6.25,4.69,0.0,2.56,6.17,0.00,...,5.62,4.70,5.36,7.29,6.39,6.51,6.08,5.53,5.92,7.75
Pup_2-cecal_contents_53_1C_Pup_2,Arm 1 (Pre-weaning P. copri colonization),2,male,5.33,6.04,4.20,0.0,2.35,5.87,1.53,...,5.29,4.31,5.15,6.90,6.11,6.46,5.58,5.44,5.83,7.48
Pup_3-cecal_contents_53_1C_Pup_3,Arm 1 (Pre-weaning P. copri colonization),3,male,6.22,6.89,4.74,0.0,2.35,6.28,2.14,...,5.39,5.68,5.92,7.39,6.22,6.52,5.95,5.58,6.20,7.82
Pup_4-cecal_contents_53_1C_Pup_4,Arm 1 (Pre-weaning P. copri colonization),4,male,6.28,6.94,4.82,0.0,2.63,6.36,0.00,...,5.61,5.53,6.02,7.33,6.50,6.48,5.87,5.51,6.16,7.85
Pup_5-cecal_contents_53_1C_Pup_5,Arm 1 (Pre-weaning P. copri colonization),5,female,6.44,7.10,5.37,0.0,3.52,6.72,2.37,...,5.65,5.70,6.33,7.34,6.55,6.51,5.92,4.33,6.23,7.91
Pup_6-cecal_contents_53_1C_Pup_6,Arm 1 (Pre-weaning P. copri colonization),6,male,6.66,7.30,5.10,0.0,3.20,6.38,2.31,...,5.91,5.89,6.28,7.36,6.53,6.70,5.94,4.83,6.61,8.03
Pup_7-cecal_contents_53_1C_Pup_7,Arm 1 (Pre-weaning P. copri colonization),7,male,6.34,6.96,4.89,0.0,3.34,6.70,2.52,...,5.50,5.05,5.99,7.66,6.52,6.93,6.20,5.27,6.54,8.04
Pup_8-cecal_contents_53_1C_Pup_8,Arm 1 (Pre-weaning P. copri colonization),8,female,6.21,6.93,5.61,0.0,2.86,6.45,2.57,...,5.70,5.12,6.06,7.53,2.79,6.91,6.59,5.40,6.85,8.07
Pup_1-cecal_contents_53_1B_Pup_1,Arm 2 (Post-weaning P. copri colonization),1,male,5.52,6.46,0.00,0.0,2.75,2.92,1.75,...,5.58,5.04,3.30,6.97,5.93,6.48,5.78,5.59,6.20,7.51


In [75]:
ABUNDANCE_STRAIN_ABBREVS = ["Bbr","Bca","Bli2D9","Bli463","Blu","Rob","Dfo","Dlo","Eav","Eco","FprB","Lga4B6","Lru","Mmu","Pco","Pst",
                  "Rgn","Rto","Sga","Spa"]

cecal_abundance_path = "formatted_input/cecal_abundance.csv"
cecal_abundance_df = pd.read_csv(cecal_abundance_path,sep=",",index_col="MouseID")
cecal_abundance_df.columns = [col.strip() for col in cecal_abundance_df.columns]


# cecal_abundance_test = log10_to_raw_abundance(cecal_abundance_df,ABUNDANCE_SCALING_FACTOR,
#                                               abundance_floor=0,drop_samples_re="1B")

cecal_abundance_E6 = GL_load.log10_to_raw_abundance(cecal_abundance_df,abundance_scaling_factor=10**6,
                                           abundance_floor=0,drop_samples_re="",
                                            abundance_strain_abbrevs=ABUNDANCE_STRAIN_ABBREVS)
cecal_abundance_E7 = GL_load.log10_to_raw_abundance(cecal_abundance_df,abundance_scaling_factor=10**7,
                                           abundance_floor=0,drop_samples_re="",
                                            abundance_strain_abbrevs=ABUNDANCE_STRAIN_ABBREVS)
cecal_abundance_E6_floor = GL_load.log10_to_raw_abundance(cecal_abundance_df,abundance_scaling_factor=10**6,
                                           abundance_floor=10**4,drop_samples_re="",
                                            abundance_strain_abbrevs=ABUNDANCE_STRAIN_ABBREVS)
cecal_abundance_E7_floor = GL_load.log10_to_raw_abundance(cecal_abundance_df,abundance_scaling_factor=10**7,
                                           abundance_floor=10**4,drop_samples_re="",
                                            abundance_strain_abbrevs=ABUNDANCE_STRAIN_ABBREVS)

with pd.option_context('display.max_columns',None):
    display(cecal_abundance_E6)
    display(cecal_abundance_E6_floor)

cecal_abundance_dir = "{0}/cecal_abundance".format(full_dir_path)
if not os.path.exists(cecal_abundance_dir):
    os.makedirs(cecal_abundance_dir)
    
cecal_abundance_E7_floor.to_csv("{0}/cecal_abundance_E7_floored.csv".format(cecal_abundance_dir))
cecal_abundance_E6_floor.to_csv("{0}/cecal_abundance_E6_floored.csv".format(cecal_abundance_dir))
cecal_abundance_E7.to_csv("{0}/cecal_abundance_E7.csv".format(cecal_abundance_dir))
cecal_abundance_E6.to_csv("{0}/cecal_abundance_E6.csv".format(cecal_abundance_dir))

,Bbr,Bca,Bli2D9,Bli463,Blu,Rob,Dfo,Dlo,Eav,Eco,FprB,Lga4B6,Lru,Mmu,Pco,Pst,Rgn,Rto,Sga,Spa
MouseID,,,,,,,,,,,,,,,,,,,,
Pup_1-cecal_contents_53_1C_Pup_1,0.912011,1.778279,0.048978,0.0,0.000363,1.479108,0.000000,1.548817,0.691831,21.379621,0.000000,0.416869,0.050119,0.229087,19.498446,2.454709,3.235937,1.202264,0.338844,0.831764
Pup_2-cecal_contents_53_1C_Pup_2,0.213796,1.096478,0.015849,0.0,0.000224,0.741310,0.000034,2.398833,0.616595,11.748976,0.000035,0.194984,0.020417,0.141254,7.943282,1.288250,2.884032,0.380189,0.275423,0.676083
Pup_3-cecal_contents_53_1C_Pup_3,1.659587,7.762471,0.054954,0.0,0.000224,1.905461,0.000138,4.168694,1.023293,14.791084,0.000071,0.245471,0.478630,0.831764,24.547089,1.659587,3.311311,0.891251,0.380189,1.584893
Pup_4-cecal_contents_53_1C_Pup_4,1.905461,8.709636,0.066069,0.0,0.000427,2.290868,0.000000,3.630781,1.258925,20.417379,0.000000,0.407380,0.338844,1.047129,21.379621,3.162278,3.019952,0.741310,0.323594,1.445440
Pup_5-cecal_contents_53_1C_Pup_5,2.754229,12.589254,0.234423,0.0,0.003311,5.248075,0.000234,4.168694,1.548817,21.379621,0.000000,0.446684,0.501187,2.137962,21.877616,3.548134,3.235937,0.831764,0.021380,1.698244
Pup_6-cecal_contents_53_1C_Pup_6,4.570882,19.952623,0.125893,0.0,0.001585,2.398833,0.000204,8.317638,2.630268,28.840315,0.000000,0.812831,0.776247,1.905461,22.908677,3.388442,5.011872,0.870964,0.067608,4.073803
Pup_7-cecal_contents_53_1C_Pup_7,2.187762,9.120108,0.077625,0.0,0.002188,5.011872,0.000331,5.370318,1.737801,21.379621,0.000000,0.316228,0.112202,0.977237,45.708819,3.311311,8.511380,1.584893,0.186209,3.467369
Pup_8-cecal_contents_53_1C_Pup_8,1.621810,8.511380,0.407380,0.0,0.000724,2.818383,0.000372,7.079458,2.238721,38.904514,0.000000,0.501187,0.131826,1.148154,33.884416,0.000617,8.128305,3.890451,0.251189,7.079458
Pup_1-cecal_contents_53_1B_Pup_1,0.331131,2.884032,0.000000,0.0,0.000562,0.000832,0.000056,0.000912,1.380384,11.220185,0.000000,0.380189,0.109648,0.001995,9.332543,0.851138,3.019952,0.602560,0.389045,1.584893


,Bbr,Bca,Bli2D9,Bli463,Blu,Rob,Dfo,Dlo,Eav,Eco,FprB,Lga4B6,Lru,Mmu,Pco,Pst,Rgn,Rto,Sga,Spa
MouseID,,,,,,,,,,,,,,,,,,,,
Pup_1-cecal_contents_53_1C_Pup_1,0.912011,1.778279,0.048978,0.0,0.01,1.479108,0.00,1.548817,0.691831,21.379621,0.00,0.416869,0.050119,0.229087,19.498446,2.454709,3.235937,1.202264,0.338844,0.831764
Pup_2-cecal_contents_53_1C_Pup_2,0.213796,1.096478,0.015849,0.0,0.01,0.741310,0.01,2.398833,0.616595,11.748976,0.01,0.194984,0.020417,0.141254,7.943282,1.288250,2.884032,0.380189,0.275423,0.676083
Pup_3-cecal_contents_53_1C_Pup_3,1.659587,7.762471,0.054954,0.0,0.01,1.905461,0.01,4.168694,1.023293,14.791084,0.01,0.245471,0.478630,0.831764,24.547089,1.659587,3.311311,0.891251,0.380189,1.584893
Pup_4-cecal_contents_53_1C_Pup_4,1.905461,8.709636,0.066069,0.0,0.01,2.290868,0.00,3.630781,1.258925,20.417379,0.00,0.407380,0.338844,1.047129,21.379621,3.162278,3.019952,0.741310,0.323594,1.445440
Pup_5-cecal_contents_53_1C_Pup_5,2.754229,12.589254,0.234423,0.0,0.01,5.248075,0.01,4.168694,1.548817,21.379621,0.00,0.446684,0.501187,2.137962,21.877616,3.548134,3.235937,0.831764,0.021380,1.698244
Pup_6-cecal_contents_53_1C_Pup_6,4.570882,19.952623,0.125893,0.0,0.01,2.398833,0.01,8.317638,2.630268,28.840315,0.00,0.812831,0.776247,1.905461,22.908677,3.388442,5.011872,0.870964,0.067608,4.073803
Pup_7-cecal_contents_53_1C_Pup_7,2.187762,9.120108,0.077625,0.0,0.01,5.011872,0.01,5.370318,1.737801,21.379621,0.00,0.316228,0.112202,0.977237,45.708819,3.311311,8.511380,1.584893,0.186209,3.467369
Pup_8-cecal_contents_53_1C_Pup_8,1.621810,8.511380,0.407380,0.0,0.01,2.818383,0.01,7.079458,2.238721,38.904514,0.00,0.501187,0.131826,1.148154,33.884416,0.010000,8.128305,3.890451,0.251189,7.079458
Pup_1-cecal_contents_53_1B_Pup_1,0.331131,2.884032,0.000000,0.0,0.01,0.010000,0.01,0.010000,1.380384,11.220185,0.00,0.380189,0.109648,0.010000,9.332543,0.851138,3.019952,0.602560,0.389045,1.584893


In [ ]:
#Ileal abundance
ileal_abundance_path = "formatted_input/ileal_abundance.csv"
ileal_abundance_df = pd.read_csv(ileal_abundance_path,sep=",",index_col="MouseID")
ileal_abundance_df.columns = [col.strip() for col in ileal_abundance_df.columns]


# ileal_abundance_test = log10_to_raw_abundance(ileal_abundance_df,ABUNDANCE_SCALING_FACTOR,
#                                               abundance_floor=0,drop_samples_re="1B")

ileal_abundance_E6 = log10_to_raw_abundance(ileal_abundance_df,abundance_scaling_factor=10**6,
                                           abundance_floor=0,drop_samples_re="1B")
ileal_abundance_E7 = log10_to_raw_abundance(ileal_abundance_df,abundance_scaling_factor=10**7,
                                           abundance_floor=0,drop_samples_re="1B")
ileal_abundance_E6_floor =  log10_to_raw_abundance(ileal_abundance_df,abundance_scaling_factor=10**6,
                                           abundance_floor=10**4,drop_samples_re="1B")
ileal_abundance_E7_floor =  log10_to_raw_abundance(ileal_abundance_df,abundance_scaling_factor=10**7,
                                           abundance_floor=10**4,drop_samples_re="1B")


with pd.option_context('display.max_columns',None):
    display(ileal_abundance_E7)
#     display(ileal_abundance_E7_floor)

ileal_abundance_dir = "{0}/ileal_abundance".format(full_dir_path)
if not os.path.exists(ileal_abundance_dir):
    os.makedirs(ileal_abundance_dir)
    
ileal_abundance_E7_floor.to_csv("{0}/ileal_abundance_E7_floored.csv".format(ileal_abundance_dir))
ileal_abundance_E6_floor.to_csv("{0}/ileal_abundance_E6_floored.csv".format(ileal_abundance_dir))
ileal_abundance_E7.to_csv("{0}/ileal_abundance_E7.csv".format(ileal_abundance_dir))
ileal_abundance_E6.to_csv("{0}/ileal_abundance_E6.csv".format(ileal_abundance_dir))

In [ ]:
def vc_strain_locus_tag(expr_df,mcseed_df):
    """Returns a DataFrame indexed on locus tags for each strain, containing columns: 
        "Filtered Loci" - number of loci corresponding to that locus tag (ie loci per strain)
        "Strain" - full strain name corresponding to locus tag 
        :param pd.DataFrame expr_df: DataFrame indexed by ORFs containing (transformed) expression data, columns
        are samples 
        :param pd.DataFrame mcseed_df: DataFrame containing mcSEED annotations for loci, not necessarily for 
        all loci in expr_df 
    """
    vc_by_strain_locus_tag = expr_df.index.str.extract(r'(\w+)_\d+',expand=False).value_counts()
#     vc_by_strain_locus_tag.drop("ROSSTS7063_a2",inplace=True) #2nd3rd_trial specific error 
    locus_tag_strains = [mcseed_df.loc[mcseed_df.index.str.contains(lt),"Isolate name"].values[0] 
                             for lt in vc_by_strain_locus_tag.index] #if lt != "ROSSTS7063_a2"]
    locus_vc_df = pd.DataFrame(index=vc_by_strain_locus_tag.index,columns=["Filtered Loci","Strain"])
    locus_vc_df.loc[:,"Filtered Loci"] = vc_by_strain_locus_tag
    locus_vc_df.loc[:,"Strain"] = locus_tag_strains
    locus_vc_df.loc[:,"Abbreviation"] = [STRAIN_TAGS[lts] for lts in locus_tag_strains]
    return locus_vc_df

def abundance_correct_expr_df(expr_df,abundance_df,locus_vc_df,dtype='float'):
    abundance_corrected = expr_df.copy()
    abundance_corrected = abundance_corrected.loc[:,abundance_corrected.columns.isin(abundance_df.index)]
    abundance_corrected.loc[:,"Locus Tag"] = abundance_corrected.index.str.extract(r'(\w+)_\d+',expand=False)
    for lt in abundance_corrected["Locus Tag"].unique():
        strain_abbrev = locus_vc_df.loc[lt,"Abbreviation"]
        lt_expr_df = abundance_corrected.loc[abundance_corrected.loc[:,"Locus Tag"]==lt]
        abundance_data = abundance_df.loc[:,strain_abbrev]
        samples = list(abundance_data.index)
        lt_ac = lt_expr_df.loc[:,lt_expr_df.columns.isin(samples)]/abundance_data
        abundance_corrected.loc[lt_ac.index,samples] = lt_ac.loc[:,samples]
    abundance_corrected.drop(columns="Locus Tag",inplace=True)
#     display(abundance_corrected.loc[np.isinf(abundance_corrected).any(axis=1),:])
    abundance_corrected.replace([np.inf, -np.inf], np.nan, inplace=True)

    abundance_corrected.replace(np.nan,0,inplace=True)
    if dtype == 'float':
        abundance_corrected = abundance_corrected.astype('float')
    elif dtype == 'int':
        abundance_corrected_expr_int = np.floor(abundance_corrected_expr)
        abundance_corrected_expr_int = abundance_corrected_expr_int.astype('int')
    return abundance_corrected
    
locus_vc_df = vc_strain_locus_tag(count_df,mcseed_df)
# display(locus_vc_df)
abundance_corrected_expr = abundance_correct_expr_df(count_df,cecal_abundance_converted,locus_vc_df)
assert(len(count_df.loc[count_df.isna().any(axis=1)])==0)
assert(len(abundance_corrected_expr.loc[abundance_corrected_expr.isna().any(axis=1)])==0)
abundance_corrected_expr_int = np.floor(abundance_corrected_expr)
abundance_corrected_expr_int = abundance_corrected_expr_int.astype('int')
display(abundance_corrected_expr)
display(abundance_corrected_expr_int)

filt_tpm = tpm_df.loc[:,tpm_df.columns.isin(abundance_corrected_expr.columns)]
    
for abundance_data,tag in [(cecal_abundance_E6,"E6"),(cecal_abundance_E7,"E7")]:
    ac_expr_data = abundance_correct_expr_df(count_df,abundance_data,locus_vc_df) 
    ac_expr_int = np.floor(ac_expr_data).astype(int)
    filt_tpm = tpm_df.loc[:,tpm_df.columns.isin(ac_expr_data.columns)]
    
    if tag == "E6":
        ac_float_E6 = ac_expr_data
        ac_int_E6 = ac_expr_int
    elif tag == "E7":
        ac_float_E7 = ac_expr_data
        ac_int_E7 = ac_expr_int
    
    if ABUNDANCE_FLOOR:
        path_tag = tag + "_floor"
    else: 
        path_tag = tag
    full_dir_path = "formatted_output/4th_trial/full/".format(path_tag)
#     if not os.path.exists(full_dir_path):
#         os.makedirs(full_dir_path)
    AC_tag_dir = "{0}/{1}".format(full_dir_path,path_tag)
    if not os.path.exists(AC_tag_dir):
        os.makedirs(AC_tag_dir)
    if overwrite_files or not os.path.exists("{0}/abundance_corrected_count.csv".format(AC_tag_dir)):
        ac_expr_data.to_csv("{0}/abundance_corrected_count.csv".format(AC_tag_dir))
        ac_expr_int.to_csv("{0}/abundance_corrected_count_int.csv".format(AC_tag_dir))
        filt_tpm.to_csv("{0}/column_filtered_tpm.csv".format(AC_tag_dir))

In [ ]:
#Float AC counts
print("==E6 float==")
reads_distribution_QC(ac_float_E6,mcseed_df,pco_col_pat='1C',no_pco_col_pat='2B')
print("==E7 float==")
reads_distribution_QC(ac_float_E7,mcseed_df,pco_col_pat='1C',no_pco_col_pat='2B')

In [ ]:
#Int AC counts
print("==E6 int==")
reads_distribution_QC(ac_int_E6,mcseed_df,pco_col_pat='1C',no_pco_col_pat='2B')
print("==E7 int==")
reads_distribution_QC(ac_int_E7,mcseed_df,pco_col_pat='1C',no_pco_col_pat='2B')

In [ ]:
display(cecal_abundance_E6)

In [ ]:
edger_filtered_counts_fpath = "formatted_output/4th_trial/edgeR_filtered/edgeR_filtered_count.csv"
edger_filtered_counts = pd.read_csv(edger_filtered_counts_fpath,index_col=0)
edger_filtered_counts.index.name = "target_id"
edger_filtered_counts.columns = [col.replace(".","-") for col in edger_filtered_counts.columns]
# display(edger_filtered_counts)
edgeR_AC_expr = abundance_correct_expr_df(edger_filtered_counts,cecal_abundance_E6,locus_vc_df)

edgeR_AC_int = np.ceil(edgeR_AC_expr).astype(int)
# display(edgeR_AC_expr)
display(edgeR_AC_int)

edgeR_AC_int.to_csv("formatted_output/4th_trial/edgeR_filtered/edgeR_filt_AC.csv")

In [ ]:
def test_area():
    cecal_abundance_df = pd.read_csv(cecal_abundance_path,sep=",",index_col="MouseID")
    cecal_abundance_df.columns = [col.strip() for col in cecal_abundance_df.columns]
    cecal_abundance_df = cecal_abundance_df.rename(columns=abbrevs_map)
    cecal_abundance_converted  = cecal_abundance_df.copy()
    cecal_abundance_converted = cecal_abundance_converted.replace("ND",0)
    cecal_abundance_converted.loc[:,STRAIN_ABBREVS] = cecal_abundance_converted.loc[:,STRAIN_ABBREVS].astype(float)
    cecal_abundance_converted.index.name="test"
    
    for col in STRAIN_ABBREVS:
        cecal_abundance_converted.loc[:,col] = np.power([10]*len(cecal_abundance_converted),
                                                        cecal_abundance_converted.loc[:,col].values)
        cecal_abundance_converted.loc[(cecal_abundance_converted.loc[:,col] < 10**4) & 
                                      (cecal_abundance_converted.loc[:,col] != 1),col] = 10**4
        #Replace 1 (from 10^0 abundance) with 0 
        cecal_abundance_converted.loc[:,col] = cecal_abundance_converted.loc[:,col].replace(1,0) 
        cecal_abundance_converted.loc[:,col] = cecal_abundance_converted.loc[:,col]/(ABUNDANCE_SCALING_FACTOR)
#     display(cecal_abundance_converted)
    
# test_area()
# display(cecal_abundance_converted)

In [ ]:
#ileal abundance
ileal_abundance_fpath = "formatted_input/ileal_abundance.csv"
ileal_abundance_df = pd.read_csv(ileal_abundance_fpath)
display(ileal_abundance_df)